## **EXTRAER DATOS DE BALANCES DE EMPRESAS DEL INDICE S&P500**

**Nota Importante: Todos los ejemplos que vamos a ver a lo largo de este cuaderno son solo para fines educativos y no constituyen recomendación de inversión.**

En este módulo aprenderemos a obtener grandes volúmenes de datos financieros relacionados con el área de pérdidas y ganancias.

Para ello, utilizaremos la librería `yfinance`. Para instalarla, sigue estos pasos:

1. Haz clic en **Terminal** (en el menú superior) o accede al **Command Prompt (CMD)** / **Análisis del Sistema** desde la barra de herramientas.
2. Escribe el siguiente comando:

    ```bash
    pip install yfinance
    ```

**Nota importante:** Las librerías de Python se actualizan con frecuencia, lo que puede hacer que algunas de sus funciones queden obsoletas. Es importante mantenerlas actualizadas. Para actualizar una librería, utiliza el siguiente comando:

```bash
pip install yfinance --upgrade --no-cache-dir
```


Si deseas verificar la versión instalada de una librería, utiliza el siguiente comando:

```bash
pip show yfinance
```

**Estas instrucciones son aplicables a cualquier librería de Python.**

# Pasos a seguir:

1. Utilizar el listado de empresas cotizadas del S&P 500 creado en el módulo anterior para acceder a la librería `yfinance`.
2. Implementar un bucle `for` que itere sobre cada uno de los tickers.
3. Construir una tabla que consolide los datos de balance de cada empresa.

**Antes de nada importamos las librerias que vamos a necesitar para este módulo**

In [1]:

import yfinance as yf # la más importante
import pandas as pd
import requests
from datetime import datetime, timedelta
import numpy as np
import time


**1. Crear el listado de tickers**

Documentación disponible en el archivo extraer_tickers_SP500.ipynb

In [2]:
########---CARGAMOS LOS TICKERS DEL S&P 500 DESDE WIKIPEDIA---########

# URL de la página de Wikipedia que contiene la lista de empresas del S&P 500
url_sp500 = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Definimos un encabezado HTTP (User-Agent)
headers = {
    "User-Agent": "Mozilla/5.0"
}

# Realizamos la petición HTTP GET a la página web
response = requests.get(url_sp500, headers=headers)

# Extraemos el contenido HTML de la respuesta en formato texto
html = response.text

# Utilizamos pandas para leer las tablas HTML contenidas en la página
# pd.read_html busca automáticamente todas las etiquetas <table> del HTML
sp500 = pd.read_html(html)[0]

# Renombramos algunas columnas para que tengan nombres más simples e intuitivos.
sp500 = sp500.rename(
    columns={
        'Symbol': 'ticker',
        'Security': 'name',
        'GICS Sector': 'sector'
            }
        )

# Seleccionamos únicamente las columnas que nos interesan, eliminando información innecesaria
sp500_clean = sp500[['ticker', 'name', 'sector']]

# En este punto, sp500_clean contiene:
# - ticker: símbolo bursátil de la empresa
# - name: nombre de la empresa
# - sector: sector económico al que pertenece

# Creamos la lista de tickers a partir de la columna 'ticker' del DataFrame limpio, añadimos unique() para evitar duplicados y tolist() para convertir el resultado en una lista de Python
tickers_list = sp500_clean['ticker'].unique().tolist()

print(tickers_list)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APP', 'APTV', 'ACGL', 'ADM', 'ARES', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'XYZ', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CPT', 'CPB', 'COF', 'CAH', 'CCL', 'CARR', 'CVNA', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'FIX', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRH', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 

C:\Users\jsolar\AppData\Local\Temp\ipykernel_16264\3618423136.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sp500 = pd.read_html(html)[0]


**2. Conexión con YahooFinance, bucle con tickers y creación de tabla con cifras de balance**

El proceso es el siguiente:

    - Creamos una Tabla vacia a la que añadiremos los datos con cada iteración

    - Vamos a abrir un bucle en el que vamos a iterar cada ticker

    - Dentro de cada iteración, vamos a descargar los datos de balance de cada ticker, en todos los años disponibles que tengamos y lo añadiremos a la tabla
    


In [12]:
#Creamos una Tabla vacia a la que añadiremos los datos con cada iteración
df_balance = pd.DataFrame()

print("Cargando datos de balance de los tickers del S&P 500...")

#Vamos a abrir un bucle en el que vamos a iterar cada ticker, ab rimos un for loop
for ticker in tickers_list:
    #Mensaje para saber en qué ticker estamos
    print(ticker)
    #Descargamos los datos de cada ticker
    stock = yf.Ticker(ticker)
    #Dentro de cada iteración, vamos a descargar los datos de balance de cada ticker, en todos los años disponibles que tengamos y lo añadiremos a la tabla
    ### LA SIGUIENTE LINEA DE CODIGO ES LA UNICA QUE CAMBIA RESPECTO A OTROS MÓDULOS
    balance = stock.balance_sheet
    #Transponemos la tabla para que las fechas sean filas y no columnas
    balance_t = balance.T
    #hacemos unpivot para tener una tabla más manejable en la que cada una de los capitulos del balance esté en una fila y no en columnas
    balance_unpiv = balance_t.reset_index().melt(id_vars='index', var_name='Account', value_name='Value')  # Unpivoting
    #Añadimos una columna con el ticker para identificar cada empresa
    balance_unpiv['ticker'] = ticker
    #Concatenamos los datos descargados a la tabla vacia creada anteriormente
    # Este ultimo paso es crucial para ir acumulando los datos de cada ticker en un solo DataFrame
    df_balance = pd.concat([df_balance, balance_unpiv], ignore_index=True)

# Tenemos el dataframe construido, ahora renombramos las columnas para que tengan nombres más claros
# Renombramos la columna 'index' a 'date' para mayor claridad
df_balance = df_balance.rename(columns={'index': 'date'
                                        , 'Account': 'account'
                                        , 'Value': 'value'})

# Muestra del DataFrame resultante
df_balance.head()

Cargando datos de balance de los tickers del S&P 500...
MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
AON
APA
APO
AAPL
AMAT
APP
APTV
ACGL
ADM
ARES
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BAX
BDX
BRK.B
BBY
TECH
BIIB
BLK
BX
XYZ
BK
BA
BKNG
BSX
BMY
AVGO
BR
BRO
BF.B
BLDR
BG
BXP
CHRW
CDNS
CPT
CPB
COF
CAH
CCL
CARR
CVNA
CAT
CBOE
CBRE
CDW
COR
CNC
CNP
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
COIN
CL
CMCSA
FIX
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CRH
CRWD
CCI
CSX
CMI
CVS
DHR
DRI
DDOG
DVA
DAY
DECK
DE
DELL
DAL
DVN
DXCM
FANG
DLR
DG
DLTR
D
DPZ
DASH
DOV
DOW
DHI
DTE
DUK
DD
ETN
EBAY
ECL
EIX
EW
EA
ELV
EME
EMR
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ERIE
ESS
EL
EG
EVRG
ES
EXC
EXE
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FISV
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GDDY
GS
HAL
HIG

,date,account,value,ticker
0,2024-12-31,Treasury Shares Number,404562753.0,MMM
1,2023-12-31,Treasury Shares Number,391451920.0,MMM
2,2022-12-31,Treasury Shares Number,394787951.0,MMM
3,2021-12-31,Treasury Shares Number,372187578.0,MMM
4,2024-12-31,Ordinary Shares Number,539470303.0,MMM


**Nota importante:** Los datos obtenidos de `yfinance` suelen estar desestructurados, tanto en su formato como en su contenido. Las diferentes secciones del balance no están correctamente ordenadas ni clasificadas, sino que aparecen de manera aleatoria.

A continuación, se muestra un ejemplo:

In [15]:
print(df_balance['account'].unique())

['Treasury Shares Number' 'Ordinary Shares Number' 'Share Issued'
 'Net Debt' 'Total Debt' 'Tangible Book Value' 'Invested Capital'
 'Working Capital' 'Net Tangible Assets' 'Capital Lease Obligations'
 'Common Stock Equity' 'Total Capitalization'
 'Total Equity Gross Minority Interest' 'Minority Interest'
 'Stockholders Equity' 'Gains Losses Not Affecting Retained Earnings'
 'Other Equity Adjustments' 'Treasury Stock' 'Retained Earnings'
 'Additional Paid In Capital' 'Capital Stock' 'Common Stock'
 'Total Liabilities Net Minority Interest'
 'Total Non Current Liabilities Net Minority Interest'
 'Other Non Current Liabilities' 'Liabilities Heldfor Sale Non Current'
 'Employee Benefits'
 'Non Current Pension And Other Postretirement Benefit Plans'
 'Tradeand Other Payables Non Current' 'Non Current Deferred Liabilities'
 'Non Current Deferred Taxes Liabilities'
 'Long Term Debt And Capital Lease Obligation'
 'Long Term Capital Lease Obligation' 'Long Term Debt'
 'Current Liabilities' 'Ot

In [14]:
#Ejemplo de cómo descargar los datos de balance de un ticker específico (por ejemplo, 'META' para Meta Platforms, Inc.)

stock = yf.Ticker('META')
#Dentro de cada iteración, vamos a descargar los datos de balance de cada ticker, en todos los años disponibles que tengamos y lo añadiremos a la tabla
balance_ticker = stock.balance_sheet
# balance_ticker.T.reset_index().melt(id_vars='index', var_name='Account', value_name='Value')  # Unpivoting
balance_ticker

,2024-12-31,2023-12-31,2022-12-31,2021-12-31
Treasury Shares Number,NaN,0.000000e+00,NaN,NaN
Ordinary Shares Number,2.534488e+09,2.561000e+09,2.614000e+09,2.741000e+09
Share Issued,2.534488e+09,2.561000e+09,2.614000e+09,2.741000e+09
Total Debt,4.906000e+10,3.723400e+10,2.659100e+10,1.387300e+10
Tangible Book Value,1.610680e+11,1.317260e+11,1.045100e+11,1.050480e+11
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,7.781500e+10,6.540300e+10,4.073800e+10,4.799800e+10
Other Short Term Investments,3.392600e+10,2.354100e+10,2.605700e+10,3.139700e+10
Cash And Cash Equivalents,4.388900e+10,4.186200e+10,1.468100e+10,1.660100e+10
Cash Equivalents,3.667100e+10,3.559700e+10,8.505000e+09,9.293000e+09


**3. Descargamos en Excel los datos**

In [13]:
## descargamos el excel sin el índice
df_balance.to_excel('balance.xlsx', index=False) 

**Fin del módulo**